<a href="https://colab.research.google.com/github/JensHOls/whisperDiarization/blob/master/WhisperDiarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# speechbrain (used for speaker embedding)
!pip install -qq torch==1.11.0 torchvision==0.12.0 torchaudio==0.11.0 torchtext==0.12.0
!pip install -qq speechbrain==0.5.12

# pyannote.audio (used for speaker diarization)
!pip install -qq pyannote.audio==2.1.1

# OpenAI whisper (used for automatic speech recognition)
!pip install -qq git+https://github.com/openai/whisper.git 

In [ ]:
# log in on Huggingface hub (where pretrained pyannote models are hosted)
from huggingface_hub import notebook_login
notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.huggingface/token
Login successful


In [ ]:
# upload an audio file
import google.colab
audio_file = list(google.colab.files.upload())[0]

Saving debat.wav to debat (1).wav


In [ ]:
# load pyannote.audio speaker diarization 
from pyannote.audio import Pipeline
speaker_diarization = Pipeline.from_pretrained("pyannote/speaker-diarization@2.1", 
                                               use_auth_token=True)

In [ ]:
# apply speaker diarization
who_speaks_when = speaker_diarization(audio_file)

In [ ]:
# load OpenAI Whisper automatic speech transcription 
import whisper

# choose among "tiny", "base", "small", "medium", "large"
# see https://github.com/openai/whisper/
model = whisper.load_model("large") 

100%|█████████████████████████████████████| 2.87G/2.87G [01:32<00:00, 33.4MiB/s]


In [ ]:
result = model.transcribe(audio_file, verbose=False, fp16=False)

Detected language: Danish


100%|██████████| 18882/18882 [30:54<00:00, 10.18frames/s]


In [48]:
import time

print(f'Language: {whisper.tokenizer.LANGUAGES[result["language"]].title()}\n')

who_speaks_when = who_speaks_when.rename_labels({"SPEAKER_00": "Moderator", "SPEAKER_01": "Løkke", "SPEAKER_02": "Støjberg"})

from pyannote.core import Segment


def format_seconds(seconds: float) -> str:
    return time.strftime("%M:%S",time.gmtime(round(seconds)))

# compare with leeway start/end of whisper and speaker segments
# this is not an optimal or reliable way to correctly match the segments
def compareWithLeeway(sentence: float, diarization: float, start: bool):
    leeway = 0.6
    if start:
        if sentence > diarization + leeway or sentence > diarization - leeway:
            return True
    else: 
        if sentence < diarization + leeway or sentence < diarization - leeway:
            return True
    return False

for segment, _, speaker in who_speaks_when.itertracks(yield_label=True):
    for sentence in result["segments"]:
        if compareWithLeeway(sentence["start"], segment.start, True) and compareWithLeeway(sentence["end"], segment.end, False):
            print(f'[{format_seconds(sentence["start"])}->{format_seconds(sentence["end"])}] {speaker}:{sentence["text"]}')

Language: Danish

[00:00->00:03] Støjberg: Jeg er også lidt overrasket over, at Lars Løkke står på den her side i aften.
[00:03->00:06] Støjberg: Jeg tænkte egentlig på, at man skulle stå ved den ledige pult dernede, fordi
[00:06->00:11] Støjberg: når man hører ham, så lyder det jo mere som om, at du kommer til at pege på Mette Frederiksen.
[00:11->00:12] Løkke: Men det er jo rent røv, de der hængere.
[00:12->00:16] Støjberg: Men det lyder jo utrolig rart, når man hører sådan noget radikal.
[00:16->00:19] Støjberg: Altså både Lars Løkke-radikal og radikal og radikal.
[00:19->00:21] Støjberg: Altså det lyder jo alle tiders.
[00:21->00:24] Støjberg: Men det er sådan lidt en drømmeverden på en eller anden måde.
[00:24->00:30] Støjberg: Og det er også derfor, at jeg er også lidt overrasket over, at Lars Løkke står på den her side i aften.
[00:30->00:33] Støjberg: Og jeg tænkte egentlig på, at man skulle stå ved den ledige pult dernede, fordi
[00:33->00:37] Støjberg: når man hører ham, så l